In [126]:
cd C:/Users/PC/Jeehyun/ECE6780/Data/vgg/

C:\Users\PC\Jeehyun\ECE6780\Data\vgg


In [127]:
import sys
import os
import h5py
import scipy.io
import numpy as np
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Lambda, Conv2DTranspose
from keras.layers import Input
from keras import applications
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import InceptionV3
from keras.models import model_from_json

In [128]:
homeDIr = 'C:/Users/PC/Jeehyun/ECE6780/Data/vgg/'
b_train = 498
m_train = 1190
b_test = 100
m_test = 139
b_val = 137
m_val = 178
nb_train_samples =  b_train + m_train
nb_validation_samples =  b_val + m_val

batch_size = 1

In [129]:
mag_fact = '200'
train_dir = 'breast_' + mag_fact + '_split/train'
val_dir = 'breast_' + mag_fact + '_split/train'

In [289]:

############################################################################################
# Nuclei_feature
############################################################################################
nuc_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/nuc_Feature/'

nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_' + mag_fact + '.mat'
nuc_data = []
with h5py.File(nuc_file_name) as f:
    for column in f['breakhis_mp_nuclei_feature']:
        row_data = []
        for row_number in range(len(column)):            
            row_data.append(f[column[row_number]][:])   
        nuc_data.append(row_data)
        
nuc_file_name = nuc_dir+ 'breakhis_mp_nuclei_feature_' + mag_fact + '_val.mat'
nuc_data_val = []
with h5py.File(nuc_file_name) as f:
    for column in f['breakhis_mp_nuclei_feature']:
        row_data = []
        for row_number in range(len(column)):            
            row_data.append(f[column[row_number]][:])   
        nuc_data_val.append(row_data)        

Nuc_feat = np.array(nuc_data)
Nuc_val_feat = np.array(nuc_data_val)
del nuc_data, nuc_data_val

Nuc_feat = Nuc_feat.reshape(len(Nuc_feat), 256,256,1)
Nuc_val_feat = Nuc_val_feat.reshape(len(Nuc_val_feat), 256,256,1)

############################################################################################
# Cascaded PCA
############################################################################################
pca_dir = 'C:/Users/PC/Jeehyun/ECE6780/Data/PCA_Feature/'

pca_file_name = pca_dir+ 'breakhis_pca_feature_' + mag_fact + '.mat'
pca_data = []
with h5py.File(pca_file_name) as f:
    for column in f['breakhis_pca_feature']:
        pca_data.append(column)
        
pca_file_name = pca_dir+ 'breakhis_pca_feature_' + mag_fact + '_val.mat'
pca_data_val = []
with h5py.File(pca_file_name) as f:
    for column in f['breakhis_pca_feature']:
        pca_data_val.append(column)
pca_data = np.array(pca_data)
pca_data_val = np.array(pca_data_val)

############################################################################################
# CNN feature
############################################################################################
vgg_train = np.load('fine_bottleneck_feat_train.npy')
vgg_val = np.load('fine_bottleneck_feat_val.npy')
vgg_train_reshape = vgg_train.reshape(1688, 7*7*512)
vgg_val_reshape = vgg_val.reshape(315, 7*7*512)


Nuc_feat_n = Nuc_feat/((Nuc_feat-Nuc_feat.min()).max())
pca_data_n = pca_data/((pca_data-pca_data.min()).max())
vgg_train_n = vgg_train_reshape/((vgg_train_reshape-vgg_train_reshape.min()).max())

Nuc_val_feat_n = Nuc_val_feat/((Nuc_val_feat-Nuc_val_feat.min()).max())
pca_data_val_n = pca_data_val/((pca_data_val-pca_data_val.min()).max())
vgg_val_n = vgg_val_reshape/((vgg_val_reshape-vgg_val_reshape.min()).max())


############################################################################################
# Input 1
############################################################################################
model_1 = Sequential()
model_1.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape= (256, 256, 1)))
model_1.add(Conv2D(64, (3, 3), activation='relu'))
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))
model_1.add(Flatten())
#model_1.add(Dense(128, activation='relu'))
#model_1.add(Dropout(0.5))
model_1.add(Dense(1, activation='softmax'))

model_1.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
model_1.summary()


############################################################################################
# Input 2
############################################################################################
PCA_input = Input(shape=(12,))

model_3 = Sequential()
model_3.add(Dense(6, input_dim=12))
model_3.add(Activation('relu'))
model_3.add(Dropout(0.25))
model_3.add(Dense(1))
model_3.add(Activation('sigmoid'))
model_3.compile(optimizer='sgd',loss='binary_crossentropy', metrics=['accuracy'])


############################################################################################
# Input 3
############################################################################################
vgg_input = Input(shape=(vgg_train.shape[1:]))
vgg_re_input = Input(shape=(vgg_train_reshape.shape[1:]))

############################################################################################
# Merging
############################################################################################
merge_layer = concatenate([model_1.output, model_3.output, vgg_re_input])
dnn_layer = Dense(128, activation="relu")(merge_layer)
dnn_output = Dense(1, activation="sigmoid")(dnn_layer)

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 254, 254, 32)      320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 252, 252, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 126, 126, 64)      0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 1016064)           0         
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 1016065   
Total params: 1,034,881
Trainable params: 1,034,881
Non-trainable params: 0
___________________________________________

In [290]:
model_cat = Model(inputs=[model_1.input, model_3.input, vgg_re_input], outputs=dnn_output)
model_cat.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])


In [291]:
batch_size = 64
epochs=1

# Nuc_feat_n
# pca_data_n
# vgg_train_n
# Nuc_val_feat_n
# pca_data_val_n
# vgg_val_n

model_cat.fit(x = [Nuc_feat_n, pca_data_n, vgg_train_n], y = train_labels,
             validation_data = ([Nuc_val_feat_n, pca_data_val_n, vgg_val_n], val_labels),
              batch_size=batch_size, epochs=epochs)

Train on 1688 samples, validate on 315 samples
Epoch 1/1
1688/1688 [==============================] - 155s 92ms/step - loss: 0.6915 - accuracy: 0.5865 - val_loss: 0.6918 - val_accuracy: 0.5683


In [299]:
batch_size = 24
epochs=1
model_cat.fit(x = [Nuc_feat_n, pca_data_n, vgg_train_n], y = train_labels,
             validation_data = ([Nuc_val_feat_n, pca_data_val_n, vgg_val_n], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)

model_new_name = 'Final_concat_Model_' + mag_fact
model_json = model_cat.to_json()
with open(model_new_name + '.json', "w") as json_file:
    json_file.write(model_json)

model_cat.save_weights(model_new_name + '.h5')

Train on 1688 samples, validate on 315 samples
Epoch 1/1
1688/1688 [==============================] - 158s 93ms/step - loss: 0.6056 - accuracy: 0.7050 - val_loss: 0.7390 - val_accuracy: 0.5651


In [302]:
############################################################################################
# Merging
############################################################################################
merge_layer = concatenate([model_1.output,  vgg_re_input])
dnn_layer = Dense(128, activation="relu")(merge_layer)
dnn_output = Dense(1, activation="sigmoid")(dnn_layer)
model_cat = Model(inputs=[model_1.input, vgg_re_input], outputs=dnn_output)
model_cat.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])



In [305]:
batch_size = 24
epochs=1
model_cat.fit(x = [Nuc_feat, vgg_train_reshape], y = train_labels,
             validation_data = ([Nuc_val_feat, vgg_val_reshape], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)


Train on 1688 samples, validate on 315 samples
Epoch 1/1
1688/1688 [==============================] - 160s 95ms/step - loss: 0.6374 - accuracy: 0.6730 - val_loss: 0.7409 - val_accuracy: 0.5651


In [314]:


print(vgg_train_reshape.max())
print(Nuc_feat.min())

Nuc_feat_n = Nuc_feat/vgg_train_reshape.max()
#pca_data_n = pca_data/((pca_data-pca_data.min()).max())
vgg_train_n = vgg_train_reshape/vgg_train_reshape.max()

Nuc_val_feat_n = Nuc_val_feat/vgg_train_reshape.max()
#pca_data_val_n = pca_data_val/((pca_data_val-pca_data_val.min()).max())
vgg_val_n = vgg_val_reshape/vgg_train_reshape.max()


22.434698
-0.604897748996685


In [322]:
batch_size = 20
epochs=3
model_cat.fit(x = [Nuc_feat, vgg_train_reshape], y = train_labels,
             validation_data = ([Nuc_val_feat, vgg_val_reshape], val_labels),
              batch_size=batch_size, epochs=epochs, shuffle=True)


Train on 1688 samples, validate on 315 samples
Epoch 1/3
1688/1688 [==============================] - 159s 94ms/step - loss: 0.5961 - accuracy: 0.7068 - val_loss: 0.7684 - val_accuracy: 0.5683
Epoch 2/3
1688/1688 [==============================] - 159s 94ms/step - loss: 0.5812 - accuracy: 0.7085 - val_loss: 0.7741 - val_accuracy: 0.5651
Epoch 3/3
1688/1688 [==============================] - 159s 94ms/step - loss: 0.5779 - accuracy: 0.7091 - val_loss: 0.7682 - val_accuracy: 0.5683


array([[-0.        , -0.        , -0.        , ..., -0.        ,
         0.03054723,  0.01296801],
       [-0.        , -0.        , -0.        , ..., -0.        ,
         0.02482537, -0.        ],
       [-0.        , -0.        ,  0.05123122, ...,  0.01144865,
         0.01608983, -0.        ],
       ...,
       [-0.        , -0.        ,  0.02423421, ..., -0.        ,
         0.02153904, -0.        ],
       [-0.        , -0.        ,  0.01688414, ..., -0.        ,
        -0.        , -0.        ],
       [ 0.00627513, -0.        , -0.        , ..., -0.        ,
         0.01011185, -0.        ]], dtype=float32)

In [307]:
model_new_name = 'Final_2_concat_Model_' + mag_fact
model_json = model_cat.to_json()
with open(model_new_name + '.json', "w") as json_file:
    json_file.write(model_json)

model_cat.save_weights(model_new_name + '.h5')